## Notes 
<b> 2/18/2022 <b>
* For SAR, only projects that have empty phase completion dates are included. 
* Projects with NO allocation amount won't appear in SAR. 
* Wants to compare cols from previous SAR with current SAR b/c she has to bold any changes in black for CTC.
* For PAP, problem is prior col is all lumped together for projects before 2020? Highlands sheet does not differentiate between the years. Wants each year to be separated out.
    
<b> To do </b>
* Make sure all the values in date columns are replaced properly. 
* Make sure grant recipients don't have multiple variations of the same recipient.
* Make sure PPNO numbers match across the sheets.
* Change col names to match the new SAR report sent over 2/15.
* SAR: allocation amount col isn't from highlands sheet, it's taken by adding GGRF Alloc + PTA-SB1. Add them back in, add the column...

In [1]:
import numpy as np
import pandas as pd
import TIRCP_functions
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.2f}".format
from siuba import *
from calitp import *


/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
E0314 21:22:16.539197288    1847 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0314 21:22:16.976509119    1847 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
#Load in Crosswalks 
FILE_NAME3 = "Allocation_PPNO_Crosswalk.csv"
allocation_ppno_crosswalk = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME3}")
    
#Allocation PPNO Crosswalk
FILE_NAME4 = "Projects_PPNO.xlsx"
project_ppno_crosswalk = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME4}")


### Testing Area

In [3]:
FILE_NAME = 'TIRCP_Allocation_March_14_2022.xlsx'
allocation = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME}")

In [4]:
allocation.columns = allocation.columns.str.strip().str.replace(' ', '_')

In [5]:
allocation.columns

Index(['Award_Year', 'Project_#', 'Grant_Recipient', 'Implementing_Agency',
       'PPNO', 'Project_ID', 'EA', 'Components', 'Phase', 'Allocation_Amount',
       'Expended_Amount', 'SB1_Funding', 'SB1_Budget_Year', 'GGRF_Funding',
       'GGRF_Budget_Year', 'CTC_Financial_Resolution',
       'CTC_Allocation_Amendment', 'CTC_Waiver', 'CalSTA_Waiver',
       'Allocation_Date', 'Completion_Date', 'PSA_#', 'CT_Document_#',
       '3rd_Party_Award_Date', 'LED', 'Date_Branch_Chief_Receives_PSA',
       'Date_Regional_Coordinator_Receives_PSA', 'Date_OC_Receives_PSA',
       'Date_OPM_Receives_PSA', 'Date_Legal_Receives_PSA',
       'Date_Returned_to_PM', 'Date_PSA_Sent_to_Local_Agency',
       'Date_PSA_Approved_by_Local_Agency', 'Date_Signed_by_DRMT',
       'PSA_Expiry_Date', 'LONP', 'Prior_Fiscal_Years_to_2020',
       'Fiscal_Year_2020-2021', 'Fiscal_Year_2021-2022',
       'Fiscal_Year_2022-2023', 'Fiscal_Year_2023-2024',
       'Fiscal_Year_2024-2025', 'Fiscal_Year_2025-2026',
       '

In [6]:
allocation=allocation.rename(columns = {'3rd_Party_Award_Date':'Third_Party_Award_Date'})

In [17]:
def allocation_function(df): 
    #FILE_NAME2 = "TIRCP_Allocation_March_14_2022.xlsx" CHANGE THIS
    #df = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME2}")CHANGE THIS
    
    #stripping spaces & _ 
    df.columns = df.columns.str.strip().str.replace(' ', '_')
    #stripping spaces in columns
    df.columns = df.columns.map(lambda x: x.strip())
    
    #drop NA
    df= df.dropna(how='all')
    ### PPNO CLEAN UP ### 
    # stripping PPNO down to <5 characters
    df = df.assign(PPNO_New = df['PPNO'].str.slice(start=0, stop=5))
    #Merge in Crosswalk 
    df = pd.merge(df, allocation_ppno_crosswalk, left_on = ["Award_Year", "Grant_Recipient"], right_on =["Award_Year","Award_Recipient"], how = "left")
    #Map Crosswalk 
    df.PPNO_New = df.apply(lambda x: x.PPNO_New if (str(x.PPNO_New2) == 'nan') else x.PPNO_New2, axis=1)
    #Drop old PPNO 
    df = df.drop(['PPNO','PPNO_New2'], axis=1).rename(columns = {'PPNO_New': 'PPNO'}) 
    
    ### DATES CLEAN UP ###
    #rename thid party award date
    df = df.rename(columns = {'3rd_Party_Award_Date':'Third_Party_Award_Date'})
    #clean up dates in a loop
    alloc_dates = ["Allocation_Date", "Third_Party_Award_Date", "Completion_Date", "LED",
                  ]
    for i in [alloc_dates]:
        df[i] = (df[i].replace('/', '-', regex = True).replace('Complete', '', regex = True)
            .replace('\n', '', regex=True).replace('Pending','TBD',regex= True)
            .fillna('TBD')
        )
    #replacing values for date columns to be coerced later 
    df["Allocation_Date"] = (df["Allocation_Date"].replace({"08/12//20": '2020-08-12 00:00:00', 
                                        
                                         'FY 20/21': '2020-12-31 00:00:00'})) 
   
    df["Completion_Date"] = (df["Completion_Date"].replace({
    'Complete\n6/1/2019': '2019-06-01 00:00:00',
    'Complete\n2/11/2018': '2018-02-11 00:00:00',
    'Complete\n6/30/2020': '2020-06-30 00:00:00',
     '\n6/30/2018': '2018-06-30 00:00:00', 
     '\n6/29/2020':'2020-06-29 00:00:00', 
        '\n11/1/2019': '2019-01-11 00:00:00',
    '\nJun-29\n':'2019-06-01 00:00:00',
    '6/30/2021\n12/31/2021\n10/20/2022': '2022-10-22 00:00:00',
    'Complete\n1/31/2020': '2020-01-31 00:00:00',
    'Complete\n8/30/2020': '2020-08-30 00:00:00',
    'June 24. 2024': '2024-06-01 00:00:00',  
    '11/21/2024\n7/30/2025 (Q4)': '2024-11-21 00:00:00', 
    'Jun-26': '2026-01-01 00:00:00', 
     'Jun-29': '2029-06-01 00:00:00',
    'Complete\n11/12/2019': '2019-11-12 00:00:00' , 
    'Deallocated': '', 
    'Jun-28': '2028-06-01 00:00:00',  
    'Jun-25': '2025-06-01 00:00:00', 
    'Jun-23':'2023-06-01 00:00:00', 
    'Jun-27': '2027-06-01 00:00:00',
    'Jan-25': '2025-01-01 00:00:00',
    '11-21-20247-30-2025 (Q4)':'2025-07-30 00:00:00',
    '6-30-202112-31-2021': '2021-12-31 00:00:00',
    '6-1-2019': '2019-06-01 00:00:00',
    '2-11-2018': '2018-02-11 00:00:00',
     '6-30-2020': '2020-06-30 00:00:00',
    ' 6-30-2018': '2018-06-30 00:00:00',
     '6-29-2020': '2020-06-29 00:00:00',
     '11-1-2019': '2019-11-01 00:00:00',
     ' 12-10-2018': '2018-12-10 00:00:00',
     ' 11-13-2019': '2019-11-13 00:00:00',
     '3-30-2020':'2020-03-30 00:00:00',
    ' 6-30-2020': '2020-06-30 00:00:00',
    '11-12-2019': '2019-11-12 00:00:00',
    '1-31-2020': '2020-01-31 00:00:00',
    '8-30-2020': '2020-08-30 00:00:00',
    '5-16-2020': '2020,05-16 00:00:00',
     '5-7-2020': '2020-05-07 00:00:00'})) 
    
    df["Third_Party_Award_Date"] = df["Third_Party_Award_Date"].replace({ 
    '-': 'TBD',
    'Pending 6/30/2022':'2022-06-30 00:00:00',
    'Augsut 12, 2021': '2021-08-12 00:00:00',})
    
    # coerce to dates
    df = df.assign(
    Allocation_Date_New = pd.to_datetime(df.Allocation_Date, errors="coerce").dt.date,
    Third_Party_Award_Date_New = pd.to_datetime(df.Third_Party_Award_Date, errors="coerce").dt.date,
    Completion_Date_New = pd.to_datetime(df.Completion_Date, errors="coerce").dt.date,
    LED_New = pd.to_datetime(df.LED, errors="coerce").dt.date)
    #dropping old date columns
    df = df.drop(alloc_dates, axis=1)
    #rename coerced columns
    df = (df.rename(columns = {'Allocation_Date_New':'Allocation_Date',
                               'Third_Party_Award_Date_New':'Third_Party_Award_Date',
                               'Completion_Date_New': 'Completion_Date','LED_New': 'LED'})
         )
    
    ### CLEAN UP MONETARY COLS ###
    # correcting string to 0 
    df["Expended_Amount"].replace({'Deallocation': 0}, inplace=True)
    #replacing monetary amounts with 0 & coerce to numeric 
    allocation_monetary_cols = ['SB1_Funding','Expended_Amount','Allocation_Amount',
       'GGRF_Funding','Prior_Fiscal_Years_to_2020',
       'Fiscal_Year_2020-2021', 'Fiscal_Year_2021-2022',
       'Fiscal_Year_2022-2023', 'Fiscal_Year_2023-2024',
       'Fiscal_Year_2024-2025', 'Fiscal_Year_2025-2026',
       'Fiscal_Year_2026-2027', 'Fiscal_Year_2027-2028',
       'Fiscal_Year_2028-2029', 'Fiscal_Year_2029-2030']
    df[allocation_monetary_cols] = df[allocation_monetary_cols].fillna(value=0)
    df[allocation_monetary_cols] = df[allocation_monetary_cols].apply(pd.to_numeric, errors='coerce')
    #rename columns that are similar to project sheet to avoid confusion
    df = (df.rename(columns = {'Allocation_Amount':'Allocation_Amt_Allocation_Sheet',
                               'Expended_Amount': 'Expended_Amt_Allocation_Sheet'})
         )
   
    return df

In [18]:
alloc_test = allocation_function(allocation)

In [19]:
alloc_test.isna().sum()

Award_Year                                  2
Project_#                                   2
Grant_Recipient                             2
Implementing_Agency                         2
Project_ID                                  7
EA                                         10
Components                                  2
Phase                                       2
Allocation_Amt_Allocation_Sheet             0
Expended_Amt_Allocation_Sheet               0
SB1_Funding                                 1
SB1_Budget_Year                            60
GGRF_Funding                                0
GGRF_Budget_Year                           43
CTC_Financial_Resolution                    9
CTC_Allocation_Amendment                  186
CTC_Waiver                                208
CalSTA_Waiver                             197
PSA_#                                       8
CT_Document_#                              33
Date_Branch_Chief_Receives_PSA            209
Date_Regional_Coordinator_Receives

### Loading in Scripts

In [10]:
test_SAR = TIRCP_functions.semi_annual_report()

In [11]:
#test_program = TIRCP_functions.program_allocation_plan()

In [12]:
alloc_test = TIRCP_functions.allocation()

In [13]:
alloc_test.columns

Index(['Award_Year', 'Project_#', 'Grant_Recipient', 'Implementing_Agency',
       'Project_ID', 'EA', 'Components', 'Phase',
       'Allocation_Amt_Allocation_Sheet', 'Expended_Amt_Allocation_Sheet',
       'SB1_Funding', 'SB1_Budget_Year', 'GGRF_Funding', 'GGRF_Budget_Year',
       'CTC_Financial_Resolution', 'CTC_Allocation_Amendment', 'CTC_Waiver',
       'CalSTA_Waiver', 'PSA_#', 'CT_Document_#',
       'Date_Branch_Chief_Receives_PSA',
       'Date_Regional_Coordinator_Receives_PSA', 'Date_OC_Receives_PSA',
       'Date_OPM_Receives_PSA', 'Date_Legal_Receives_PSA',
       'Date_Returned_to_PM', 'Date_PSA_Sent_to_Local_Agency',
       'Date_PSA_Approved_by_Local_Agency', 'Date_Signed_by_DRMT',
       'PSA_Expiry_Date', 'LONP', 'Prior_Fiscal_Years_to_2020',
       'Fiscal_Year_2020-2021', 'Fiscal_Year_2021-2022',
       'Fiscal_Year_2022-2023', 'Fiscal_Year_2023-2024',
       'Fiscal_Year_2024-2025', 'Fiscal_Year_2025-2026',
       'Fiscal_Year_2026-2027', 'Fiscal_Year_2027-2028',


In [14]:
tableau_test = TIRCP_functions.tableau()

KeyError: "['Vendor_ID_#', 'Key_Project_Elements', 'Technical_Assistance-CALTP_(Y/N)', 'Percentge_Allocated'] not in index"

In [ ]:
project_test = TIRCP_functions.project()

In [ ]:
project_test.columns

In [ ]:
project_test

In [ ]:
#tableau_test = tableau_test.to_parquet('Tableau_parquet.parquet') 

### Once I get the final data, compare the PPNO # across the 2 sheets 

In [ ]:
PPNO_project = set(project_test.PPNO.unique().tolist())
PPNO_allocation = set(alloc_test.PPNO.unique().tolist())

In [ ]:
PPNO_project - PPNO_allocation #checking for differences